# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *

### Step 1: Scope the Project and Gather Data

#### Scope 
Purpose: To create a flying passengers data repository based on immigration, cities, airport codes data files. With this information we can get differnt query results like passengers count to visit a city in a month, most using visa type of passengers for flying..etc, so that teams can validate their exisiting policies and provide better support to people.

#### Describe and Gather Data 
Data from immigration file, airport codes file and us-cities-demographic file

In [2]:
 # connect to flying db database
conn = psycopg2.connect("host=127.0.0.1 dbname=flyingdb user=student password=student")
cur = conn.cursor()
conn.set_session(autocommit=True)

### immigration data


In [3]:
# Read the immigration data here
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [7]:
#df1 = df.iloc[ :10, :].copy()
#df1

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
5,18.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MI,20555.0,...,NaN,M,1959.0,09302016,NaN,NaN,AZ,9.247104e+10,00602,B1
6,19.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NJ,20558.0,...,NaN,M,1953.0,09302016,NaN,NaN,AZ,9.247140e+10,00602,B2
7,20.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NJ,20558.0,...,NaN,M,1959.0,09302016,NaN,NaN,AZ,9.247161e+10,00602,B2
8,21.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NY,20553.0,...,NaN,M,1970.0,09302016,NaN,NaN,AZ,9.247080e+10,00602,B2
9,22.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NY,20562.0,...,NaN,M,1968.0,09302016,NaN,NaN,AZ,9.247849e+10,00608,B1


In [4]:
#Replace all existing null values or invalid data with zeroes
df.fillna(0,inplace=True)

In [5]:
# using dictionary to convert specific columns 
convert_dict = {'cicid': int, 
                'i94yr': int,
                'i94mon' : int,
                'i94visa' : int,
                'count' : int,
                'biryear' : int
               } 
df = df.astype(convert_dict)

In [6]:
#Fetch required columns from the dataset
immigration_data = df.iloc[:,[0,1,2,5,8,11,12,20,22,27]].values.tolist()
#immigration_data[0]

[6, 2016, 4, 'XXX', 0, 2, 1, 1979, 0, 'B2']

In [7]:
#print(len(immigration_data))

3096313


In [8]:
# Insert data into the immigration table with a limitation of one million records
for i in range(1000000):
    try:
        cur.execute(dimimmigration_table_insert, immigration_data[i])
    except psycopg2.IntegrityError as e:
        conn.rollback()
        print('insert failure:',e)
        #continue
    except ValueError as e:
        conn.rollback()
        print('insert failure:',e)
        #continue
    else:
        conn.commit()

insert failure: A string literal cannot contain NUL (0x00) characters.
insert failure: A string literal cannot contain NUL (0x00) characters.
insert failure: A string literal cannot contain NUL (0x00) characters.
insert failure: A string literal cannot contain NUL (0x00) characters.
insert failure: A string literal cannot contain NUL (0x00) characters.
insert failure: A string literal cannot contain NUL (0x00) characters.
insert failure: A string literal cannot contain NUL (0x00) characters.
insert failure: A string literal cannot contain NUL (0x00) characters.
insert failure: A string literal cannot contain NUL (0x00) characters.
insert failure: A string literal cannot contain NUL (0x00) characters.
insert failure: A string literal cannot contain NUL (0x00) characters.


In [8]:
	
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.\
# config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
# .enableHiveSupport().getOrCreate()
# df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [11]:
# #write to parquet
# df_spark.write.parquet("sas_data")
# df_spark=spark.read.parquet("sas_data")

### airport codes data

In [9]:
# Read the airport codes data here
df2 = pd.read_csv('airport-codes_csv.csv')
df2.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [10]:
#Replace all existing null values or invalid data with zeroes
df2.fillna(0,inplace=True)

In [11]:
#Fetch required columns from the dataset
airports_data = df2.iloc[:,[0,2,1,6,7,8,10]].values.tolist()
#print(airports_data[0])
#print(len(airports_data))

['00A', 'Total Rf Heliport', 'heliport', 'US-PA', 'Bensalem', '00A', '00A']
55075


In [12]:
# Insert data into the airports table 
for i in range(len(airports_data)):
    try:
        cur.execute(dimairports_table_insert, airports_data[i])
    except psycopg2.IntegrityError as e:
        conn.rollback()
        print('insert failure:',e)
        #continue
    except ValueError as e:
        conn.rollback()
        print('insert failure:',e)
        #continue
    else:
        conn.commit()

### cities data

In [13]:
# Read the cities information data here
df3 = pd.read_csv('us-cities-demographics.csv',sep = ';')
df3.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [14]:
#Replace all existing null values or invalid data with zeroes
df3.fillna(0,inplace=True)

In [15]:
# using dictionary to convert specific columns 
convert_dict = {'Male Population': int, 
                'Female Population': int
               } 
df3 = df3.astype(convert_dict)

In [16]:
#Fetch required columns from the dataset
cities_data = df3.iloc[:,[0,1,9,2,3,4,5]].values.tolist()
# print(cities_data[0])
# print(len(cities_data))

['Silver Spring', 'Maryland', 'MD', 33.8, 40601, 41862, 82463]
2891


In [17]:
# Insert data into the cities table 
for i in range(len(cities_data)):
    try:
        cur.execute(dimcities_table_insert, cities_data[i])
    except psycopg2.IntegrityError as e:
        conn.rollback()
        print('insert failure:',e)
        #continue
    except ValueError as e:
        conn.rollback()
        print('insert failure:',e)
        #continue
    else:
        conn.commit()

### factflying table

In [18]:
#Fetching information from dimension tables and loading into fact tables
cur.execute(fact_select)
# results = cur.fetchone()
results = cur.fetchall()


In [20]:
for row in results:
    print(row)
    if row:
        city, identity, travelid, passengercount = row
    else:
        city, identity, travelid, passengercount = None, None, None, None
    fact_table_data = [city, identity, travelid, passengercount]
    
    try:
        cur.execute(factflying_table_insert, fact_table_data)
        print(fact_table_data)
    except psycopg2.IntegrityError as e:
        conn.rollback()
        print('insert failure:',e)
        #continue
    except ValueError as e:
        conn.rollback()
        print('insert failure:',e)
        #continue
    else:
        conn.commit()

('Abilene', 'KADS', '1280940', 1)
['Abilene', 'KADS', '1280940', 1]
('Abilene', 'KADS', '1291613', 1)
['Abilene', 'KADS', '1291613', 1]
('Abilene', 'KADS', '1291614', 1)
['Abilene', 'KADS', '1291614', 1]
('Abilene', 'KADS', '355168', 1)
['Abilene', 'KADS', '355168', 1]
('Abilene', 'KADS', '368157', 1)
['Abilene', 'KADS', '368157', 1]


### Step 2: Explore and Assess the Data
#### Explore the Data 
Data from immigration file contains passenger information along with immigration data, visa information
Data from airport codes file contains airport related information like type, state, airport code information
Data from us-cities-demographic file contains city information including population
#### Cleaning Steps
The information in above data files contains invalid data like null values but most of the required columns data is avaialble, so decided to replace all the null values with zeroes

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
With the available data in 3 different files and load into respective tables it is feasable to use relational data model as queries we apply on tables contains join and aggregatios to get the perfect results

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Execute the ETL pipeline based on execution guidelines available in Readme.md file

### Data model: Relational 
#### Schema: Star schema
<img src="DataAnalysis.jpg">

#### 4.2 Data Quality Checks
Data quality checks performed by maintaing proper constraints while creating tables and inserting data into tables
 * Integrity constraints on the relational database (e.g., primary key, data type, not null, etc.)
 * Unit tests for the scripts - you can try to execute the test.ipynb file to test the results
 * Source/Count checks to ensure completeness
 


#### 4.3 Data dictionary 
Data dictionary for the above data model contains a brief description of what the data is and where it came from. 
<img src="TableFieldDescriptions.jpg">

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

I have used relational data model compatabile data base which is PostgreSQL DB for storing and query the data from/to the tables

Data model design is completely based on an assumption that data reception is monthly basis, so execution of ETL pipeline for loading data into tables

If the data was increased by 100% then I would prefer to go with cloud basis storage, data analysis by using AWS services as it provided high scalability, reliability and availability

If we need to do this task on daily basis at particular time and to update the dash board with the exact results, I would rather prefer executing this ETL pipeline by using the airflow which controls and automates the task execution

If the data base created for air passengers information need to be accessed by different people across different locations, then I would prefer completely go with cloud services provided by AWS to give necessary access grants, permissions to the people who are accessing it. This also proovides backup (copy) of data in across all nodes in the provided cluster, so there is no outage of  data and application always be in ready condition.